In [1]:
import pandas as pd

In [2]:
# Read in raw kaggle data
train = pd.read_csv('./train.csv').drop(columns='ID')
# Unmodified training data
train

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,Bone marrow,...,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356
1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672
2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793
3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349
4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,...,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,Intermediate - TED AML case <missing cytogenetics,NaN,Favorable,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,...,Bi-directional non-permissive,NaN,"N/A, Mel not given",8.0,NaN,2.0,No,10.0,0.0,18.633
28796,High,No,Poor,Yes,1.0,4.0,No TBI,No,5.0,Peripheral blood,...,GvH non-permissive,Related,"N/A, Mel not given",6.0,Yes,1.0,Yes,8.0,1.0,4.892
28797,TBD cytogenetics,NaN,Poor,NaN,2.0,8.0,No TBI,NaN,6.0,Peripheral blood,...,GvH non-permissive,Unrelated,"N/A, Mel not given",8.0,NaN,2.0,No,10.0,0.0,23.157
28798,N/A - non-malignant indication,No,Poor,No,1.0,4.0,No TBI,No,3.0,Peripheral blood,...,Permissive mismatched,Related,MEL,4.0,No,1.0,No,5.0,0.0,52.351


In [3]:
# Encode non-numeric features (just treat NaN as another value)
objs = train.select_dtypes('object')
for col in objs.columns:
    column = objs[col]
    unique = column.unique()
    i = 0
    for value in unique:
        objs[col] = objs[col].replace(value, i)
        i += 1
# Rejoin encoded object df with numeric columns
train = pd.concat([objs, train.select_dtypes('number')], axis=1)
# train

C:\Users\Scott\AppData\Local\Temp\ipykernel_8924\2782155656.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  objs[col] = objs[col].replace(value, i)


In [4]:
# Replace NaN with -1 to signal missing value to NN
# I picked this method after a little research, but this method could be changed.
train = train.fillna(-1)
train

,dri_score,psych_disturb,cyto_score,diabetes,tbi_status,arrhythmia,graft_type,vent_hist,renal_issue,pulm_severe,...,age_at_hct,hla_match_a_low,hla_match_b_high,comorbidity_score,karnofsky_score,hla_low_res_8,hla_match_drb1_high,hla_low_res_10,efs,efs_time
0,0,0.0,0,0.0,0,0,0,0.0,0,0,...,9.942,2.0,2.0,0.0,90.0,8.0,2.0,10.0,0.0,42.356
1,1,0.0,1,0.0,1,0,1,0.0,0,0,...,43.705,2.0,2.0,3.0,90.0,8.0,2.0,10.0,1.0,4.672
2,0,0.0,0,0.0,0,0,0,0.0,0,0,...,33.997,2.0,2.0,0.0,90.0,8.0,2.0,10.0,0.0,19.793
3,2,0.0,1,0.0,0,0,0,0.0,0,0,...,43.245,2.0,2.0,0.0,90.0,8.0,2.0,10.0,0.0,102.349
4,2,0.0,0,0.0,0,0,1,0.0,0,0,...,29.740,2.0,2.0,1.0,90.0,8.0,2.0,10.0,0.0,16.223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,7,3.0,4,0.0,0,0,1,0.0,0,2,...,51.136,2.0,2.0,0.0,-1.0,8.0,2.0,10.0,0.0,18.633
28796,2,0.0,2,1.0,0,0,1,0.0,0,0,...,18.075,2.0,1.0,3.0,90.0,6.0,1.0,8.0,1.0,4.892
28797,6,3.0,2,3.0,0,2,1,0.0,1,2,...,51.005,2.0,2.0,5.0,90.0,8.0,2.0,10.0,0.0,23.157
28798,0,0.0,2,0.0,0,0,1,0.0,1,2,...,0.044,1.0,1.0,1.0,90.0,4.0,1.0,5.0,0.0,52.351


**Room for additional feature engineering here...**

In [5]:
# Save preprocessed data
train.to_csv('train_processed.csv')